In [6]:
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
# ============================================================
# ✅ FIX PATH FOR NOTEBOOK INSIDE /notebooks
# ============================================================
import sys, os
from pathlib import Path

# Xác định thư mục cha của notebooks/
project_root = Path(os.getcwd()).parent  # => C:\Users\User\RR_model
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))  # ⚠️ thêm project_root, KHÔNG thêm src

print("📁 Project root added:", project_root)

# Kiểm tra lại
import importlib.util
print("🔍 src module found:", importlib.util.find_spec("src"))

📁 Project root added: C:\Users\User\RR_model
🔍 src module found: ModuleSpec(name='src', loader=<_frozen_importlib_external.SourceFileLoader object at 0x00000269A87613D0>, origin='C:\\Users\\User\\RR_model\\src\\__init__.py', submodule_search_locations=['C:\\Users\\User\\RR_model\\src'])


In [8]:
# ===========================================================
# 📘 Roll Rate Demo v7 - Sử dụng dữ liệu thật từ Oracle hoặc SQL file
# ===========================================================

from pathlib import Path
import sys
import pandas as pd
import numpy as np

# --- 1️⃣ Setup đường dẫn và import các module ---
root = Path(".").resolve()
sys.path.append(str(root / "src"))

from src.data_loader import load_data
from src.config import CFG, OUT_ROOT
from src.rollrate.transition import compute_transition_from_pairs, compute_transition_by_mob

from src.rollrate.export_excel import export_transition_excel
from src.rollrate.backtest import (
    compute_actual_matrix,
    rollforward_validation,
    plot_matrix_diff,
    plot_distribution_compare
)

print("✅ Import modules thành công.")
print("Cấu hình CFG:", CFG)


# --- 2️⃣ Load dữ liệu thực tế ---
#sql_path = r"C:/Users/MAFC4709/Python_work/RR_model/sql/uns2.sql"

#df = load_data(sql_path)
df = load_data()
df['DISBURSAL_DATE'] = df['DISBURSAL_DATE'].apply(lambda x: x.replace(day=1))

#print(f"📦 Đã load {len(df):,} dòng dữ liệu từ {sql_path}")

# Kiểm tra các cột quan trọng
required_cols = [CFG["loan"], CFG["mob"], CFG["state"], CFG["ead"], CFG["cutoff"]]
missing = [c for c in required_cols if c not in df.columns]
if missing:
    raise ValueError(f"❌ Thiếu các cột cần thiết trong data: {missing}")

print("✅ Dữ liệu có đủ các cột cần thiết:", required_cols)

# Nếu chưa có PRODUCT_TYPE thì thêm mặc định
if "PRODUCT_TYPE" not in df.columns:
    df["PRODUCT_TYPE"] = "A"
    print("⚠️ Không có PRODUCT_TYPE trong data, gán mặc định = 'A'")

# Convert cutoff về datetime
df[CFG["cutoff"]] = pd.to_datetime(df[CFG["cutoff"]])





✅ Import modules thành công.
Cấu hình CFG: {'loan': 'AGREEMENT_ID', 'mob': 'MOB', 'state': 'STATE_MODEL', 'orig_date': 'DISBURSAL_DATE', 'ead': 'PRINCIPLE_OUTSTANDING', 'disb': 'DISBURSAL_AMOUNT', 'cutoff': 'CUTOFF_DATE'}
📦 Loading Parquet from: C:\Users\User\RR_model\data\parquet
✅ Loaded 25,998,763 rows via pyarrow.dataset from C:\Users\User\RR_model\data\parquet
✅ Dữ liệu có đủ các cột cần thiết: ['AGREEMENT_ID', 'MOB', 'STATE_MODEL', 'PRINCIPLE_OUTSTANDING', 'CUTOFF_DATE']


In [5]:
# ============================================================
# FULL PIPELINE 1-CELL
#  - Transition → Forecast lịch sử → Calibration (k) → Seasonality
#  - Sale plan → Apply k + season → Aggregate → Export Excel
# ============================================================

# 0) Autoreload để mỗi lần sửa module không phải restart kernel
%load_ext autoreload
%autoreload 2##

df_raw[orig_col] = pd.to_datetime(df_raw[orig_col])

print("✔ df_raw:", df_raw.shape)

# ============================================================
# STEP 1 – TRANSITION MATRICES
# ============================================================
matrices_by_mob, parent_fallback = compute_transition_by_mob(df_raw)
print("✔ matrices_by_mob built")

# ============================================================
# STEP 2 – FORECAST LỊCH SỬ (ENGINE CŨ) → LIFECYCLE BASE (EAD)
# ============================================================
forecast_results = forecast_all_vintages(
    df_raw=df_raw,
    matrices_by_mob=matrices_by_mob,
    max_mob=29,
    enable_macro=False,
)

actual_results = get_actual_all_vintages_amount(df_raw)
lifecycle_dict = combine_all_lifecycle_amount(actual_results, forecast_results)
df_lifecycle_base = lifecycle_to_long_df_amount(lifecycle_dict)

print("✔ df_lifecycle_base:", df_lifecycle_base.shape)

# ============================================================
# STEP 3 – TAG FORECAST + TÍNH DEL METRICS (CHO CALIBRATION)
# ============================================================
df_lifecycle_tagged = tag_forecast_rows_amount(df_lifecycle_base, df_raw)
df_lifecycle_for_k = add_del_metrics(df_lifecycle_tagged, df_raw)

print("✔ df_lifecycle_for_k (có DELxx_PCT):", df_lifecycle_for_k.shape)

# ============================================================
# STEP 4 – FORECAST FULL HISTORY (MARKOV THUẦN) + TÍNH k
# ============================================================
df_full_fc = forecast_full_history(
    df_raw=df_raw,
    matrices_by_mob=matrices_by_mob,
    parent_fallback=parent_fallback,
    max_mob=29,
)
print("✔ df_full_fc:", df_full_fc.shape)

k_dict = compute_k_per_product_ifrs(
    df_actual_lifecycle=df_lifecycle_for_k,
    df_full_forecast=df_full_fc,
    anchor_map=anchor_map,
    default_anchor_mob=24,
    min_obs=4,
)

print("✔ k_dict:", k_dict)

# ============================================================
# STEP 5 – ÁP k → LIFECYCLE (EAD) + TÍNH LẠI DEL METRICS
# ============================================================
df_lifecycle_calib_ead = apply_k_to_lifecycle(df_lifecycle_base, k_dict)
df_lifecycle_calib_ead = tag_forecast_rows_amount(df_lifecycle_calib_ead, df_raw)
df_lifecycle_calib = add_del_metrics(df_lifecycle_calib_ead, df_raw)

print("✔ df_lifecycle_calib:", df_lifecycle_calib.shape)

# ============================================================
# STEP 6 – SEASONALITY (THEO THÁNG GIẢI NGÂN) + ÁP VÀO LIFECYCLE
# ============================================================
df_season_source = df_lifecycle_calib[df_lifecycle_calib["IS_FORECAST"] == 0].copy()
seasonality = build_seasonality(df_season_source, smooth_window=3)
print("✔ seasonality products:", list(seasonality.keys()))

df_lifecycle_season_ead = apply_seasonality_to_lifecycle(
    df_lifecycle_calib,
    seasonality=seasonality,
)

df_lifecycle_final = add_del_metrics(df_lifecycle_season_ead, df_raw)
print("✔ df_lifecycle_final:", df_lifecycle_final.shape)

# Đảm bảo cột flag
if "IS_FORECAST" not in df_lifecycle_final.columns:
    df_lifecycle_final = tag_forecast_rows_amount(df_lifecycle_final, df_raw)
df_lifecycle_final["SOURCE"] = "Historical"
df_lifecycle_final["IS_FORECAST"] = df_lifecycle_final["IS_FORECAST"].fillna(0).astype(int)

# ============================================================
# STEP 7 – FORECAST SALE PLAN + ÁP k + SEASONALITY + DEL METRICS
# ============================================================
# 7.1 Load sale plan
sale_plan_df = pd.read_excel(SALE_PLAN_PATH, sheet_name=SALE_PLAN_SHEET)
sale_plan_df["VINTAGE_DATE"] = pd.to_datetime(sale_plan_df["VINTAGE_DATE"])

def normalize_score(x):
    if pd.isna(x):
        return "ALL"
    s = str(x).strip().upper()
    if s in ["", "NA", "NAN", "NONE", "NULL"]:
        return "ALL"
    return s

sale_plan_df["RISK_SCORE"] = sale_plan_df["RISK_SCORE"].apply(normalize_score)

print("✔ sale_plan_df:", sale_plan_df.shape)

# 7.2 Forecast EAD theo MOB cho sale plan
states = list(BUCKETS_CANON)
sale_plan_fc_ead = forecast_sale_plan_by_mob(
    sale_plan_df=sale_plan_df,
    matrices_by_mob=matrices_by_mob,
    parent_fallback=parent_fallback,
    mob_target=24,
    start_state=BUCKETS_CANON[0],
    states=states,
)
print("✔ sale_plan_fc_ead:", sale_plan_fc_ead.shape)

# 7.3 Áp k
sale_plan_fc_calib = apply_k_to_sale_plan(sale_plan_fc_ead, k_dict)

# 7.4 Áp seasonality
sale_plan_fc_season = apply_seasonality_to_sale_plan(
    sale_plan_fc_calib,
    seasonality=seasonality,
)

# 7.5 Tính DISB_TOTAL cho sale plan từ MOB0 (DPD0)
df_plan = sale_plan_fc_season.copy()

# DISB_TOTAL = tổng DPD0 tại MOB 0 per cohort (Product × Score × Vintage)
disb_plan = (
    df_plan[df_plan["MOB"] == 0]
    .groupby(["PRODUCT_TYPE", "RISK_SCORE", "VINTAGE_DATE"])["DPD0"]
    .sum()
    .rename("DISB_TOTAL")
    .reset_index()
)

df_plan = df_plan.merge(
    disb_plan,
    on=["PRODUCT_TYPE", "RISK_SCORE", "VINTAGE_DATE"],
    how="left"
)

# 7.6 Tính DEL AMT & PCT cho sale plan
df_plan["DEL30_AMT"] = df_plan[BUCKETS_30P].sum(axis=1)
df_plan["DEL60_AMT"] = df_plan[BUCKETS_60P].sum(axis=1)
df_plan["DEL90_AMT"] = df_plan[BUCKETS_90P].sum(axis=1)

denom_plan = df_plan["DISB_TOTAL"].replace(0, np.nan)

df_plan["DEL30_PCT"] = df_plan["DEL30_AMT"] / denom_plan
df_plan["DEL60_PCT"] = df_plan["DEL60_AMT"] / denom_plan
df_plan["DEL90_PCT"] = df_plan["DEL90_AMT"] / denom_plan

df_plan["IS_FORECAST"] = 1
df_plan["SOURCE"] = "SALE_PLAN"

df_plan_final = df_plan.copy()
print("✔ df_plan_final:", df_plan_final.shape)

# ============================================================
# STEP 8 – COMBINE LIFECYCLE + SALE PLAN → PRODUCT → PORTFOLIO → EXPORT
# ============================================================
# 8.1 Ghép lifecycle + sale plan
df_all = pd.concat(
    [df_lifecycle_final, df_plan_final],
    ignore_index=True
).sort_values(
    ["PRODUCT_TYPE", "RISK_SCORE", "VINTAGE_DATE", "MOB"]
)

print("✔ df_all (lifecycle + sale plan):", df_all.shape)

# 8.2 Aggregate Score → Product
df_prod = aggregate_to_product(df_all)
print("✔ df_prod (product level):", df_prod.shape)

# 8.3 Aggregate Product → Portfolio
df_port = aggregate_products_to_portfolio(
    df_del_prod=df_prod,
    portfolio_name="PORTFOLIO_ALL",
)
print("✔ df_port (portfolio level):", df_port.shape)

df_prod_port = pd.concat([df_prod, df_port], ignore_index=True)

# 8.4 actual_info: max MOB actual per Product × Vintage
actual_info_prod = (
    df_all[df_all["IS_FORECAST"] == 0]
    .groupby(["PRODUCT_TYPE", "VINTAGE_DATE"])["MOB"]
    .max()
    .to_dict()
)

actual_info = extend_actual_info_with_portfolio(
    actual_info_prod,
    portfolio_name="PORTFOLIO_ALL",
)

print("✔ actual_info entries:", len(actual_info))

# 8.5 Export Excel
output_file = "Lifecycle_All_Products_With_SalePlan_Calib_Season.xlsx"

export_lifecycle_all_products_one_file(
    df_del_prod=df_prod_port,
    actual_info=actual_info,
    filename=output_file,
)

print(f"🎉 DONE — Exported {output_file}")


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


[autoreload of src.rollrate.calibration failed: Traceback (most recent call last):
  File "C:\Users\User\anaconda3\envs\rrmodel\Lib\site-packages\IPython\extensions\autoreload.py", line 322, in check
    elif self.deduper_reloader.maybe_reload_module(m):
         ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~^^^
  File "C:\Users\User\anaconda3\envs\rrmodel\Lib\site-packages\IPython\extensions\deduperreload\deduperreload.py", line 545, in maybe_reload_module
    new_source_code = f.read()
  File "C:\Users\User\anaconda3\envs\rrmodel\Lib\encodings\cp1252.py", line 23, in decode
    return codecs.charmap_decode(input,self.errors,decoding_table)[0]
           ~~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
UnicodeDecodeError: 'charmap' codec can't decode byte 0x8d in position 3299: character maps to <undefined>
]


ImportError: cannot import name 'forecast_full_history' from 'src.rollrate.calibration' (C:\Users\User\RR_model\src\rollrate\calibration.py)

In [14]:
from src.rollrate.calibration import (
    compute_k_per_product_ifrs,
    apply_k_to_lifecycle,
    apply_k_to_sale_plan,
)
from src.rollrate.forecast import forecast_to_dataframe
from src.rollrate.calibration import extract_actual_del90, extract_forecast_del90
from src.rollrate.lifecycle import BUCKETS_90P
# 4.3 Merge actual + forecast vào dict lifecycle
lifecycle_dict = combine_all_lifecycle_amount(actual_results, forecast_results)

# 4.4 Long format EAD theo bucket
df_lifecycle = lifecycle_to_long_df_amount(lifecycle_dict)

# 4.5 THÊM DEL METRICS → tạo DEL30/60/90_AMT & DEL30/60/90_PCT
df_lifecycle = add_del_metrics(df_lifecycle, df)
#df_actual   = extract_actual_del90(df_lifecycle)
df_forecast = extract_forecast_del90(df_lifecycle)
# Tính k per product (dùng trimmed_mean)
k_dict = compute_k_per_product_ifrs(
    df_actual_lifecycle=df_lifecycle,
    df_full_forecast=df_full_fc,
    default_anchor_mob=24,
    min_obs=3,
    method="trimmed_mean",
    trim_prop=0.2,
)
print("k_dict =", k_dict)

# Áp k cho lifecycle lịch sử
df_lifecycle_calib_ead = apply_k_to_lifecycle(
    df_lifecycle=df_lifecycle_tagged,   # đã có IS_FORECAST
    k_dict=k_dict,
    default_apply_mob=24,
)

# Áp k cho SALE PLAN (tất cả MOB)
df_plan_calib_ead = apply_k_to_sale_plan(
    df_plan_fc=sale_plan_fc_ead,
    k_dict=k_dict,
)


KeyError: 'df_lifecycle cần cột IS_FORECAST để tách actual vs forecast.'

In [ ]:
#########################################

In [16]:
df_actual_lc = get_actual_all_vintages_amount(df_raw)
df_actual_lc = add_del_metrics(df_actual_lc, df_raw)
df_actual_lc["IS_FORECAST"] = 0


AttributeError: 'dict' object has no attribute 'columns'

In [ ]:
forecast_results = forecast_all_vintages(
    df_raw=df_raw,
    matrices_by_mob=matrices_by_mob,
    max_mob=30
)

df_forecast_full = lifecycle_to_long_df_amount(forecast_results)
df_forecast_full = add_del_metrics(df_forecast_full, df_raw)
df_forecast_full["IS_FORECAST"] = 1


In [ ]:
df_k_input = pd.concat(
    [df_actual_lc, df_forecast_full],
    ignore_index=True
)


In [ ]:
H_MAP_APPLY = {
    "CDLPIL": 4,
    "TWLPIL": 4,
    "SPLPIL": 4,
    "SALPIL": 5,
    "TOPUP": 6,
    "XSELL": 6,
}

k_dict = compute_k_per_product_ifrs(
    df_actual_lifecycle=df_k_input,
    df_full_forecast=df_forecast_full,
    H_map_apply=H_MAP_APPLY,
    method="trimmed_mean",
    trim_prop=0.2   # bỏ 20% hai đầu
)

print("K values:", k_dict)


In [ ]:
df_forecast_full_k = apply_k_to_lifecycle(df_forecast_full, k_dict)


In [18]:
SALE_PLAN_PATH = r"C:/Users/MAFC4709/Python_work/RR_model/sale_plan_5yF2.xlsx"        # sửa lại nếu khác
SALE_PLAN_SHEET = "SALE_PLAN"
sale_plan_df = pd.read_excel(SALE_PLAN_PATH, sheet_name=SALE_PLAN_SHEET)
sale_plan_df["VINTAGE_DATE"] = pd.to_datetime(sale_plan_df["VINTAGE_DATE"])

In [25]:
# STEP 6 – SEASONALITY (THEO THÁNG GIẢI NGÂN) + ÁP VÀO LIFECYCLE
# ============================================================
df_season_source = df_lifecycle_calib[df_lifecycle_calib["IS_FORECAST"] == 0].copy()
seasonality = build_seasonality(df_season_source, smooth_window=3)
print("✔ seasonality products:", list(seasonality.keys()))

df_lifecycle_season_ead = apply_seasonality_to_lifecycle(
    df_lifecycle_calib,
    seasonality=seasonality,
)


NameError: name 'df_lifecycle_calib' is not defined

In [24]:
from src.rollrate.seasonality import (
    build_seasonality,
    apply_seasonality_to_lifecycle,
    apply_seasonality_to_sale_plan,
)
states = list(dict.fromkeys(list(BUCKETS_CANON) + ["WRITEOFF", "PREPAY"]))
sale_plan_fc = forecast_sale_plan_by_mob(
    sale_plan_df=sale_plan_df,
    matrices_by_mob=matrices_by_mob,
    parent_fallback=parent_fallback,
    mob_target=24,
    start_state=BUCKETS_CANON[0],
    states=states,
)

sale_plan_fc = apply_k_to_sale_plan(
    df_plan_fc=sale_plan_fc,
    k_dict=k_dict,
)

sale_plan_fc = apply_seasonality_to_sale_plan(
    df_plan_fc=sale_plan_fc,
    seasonality=seasonality,
)

# ✅ dùng hàm riêng cho sale plan, KHÔNG đụng add_del_metrics(df_raw)
sale_plan_fc = add_del_metrics_for_sale_plan(
    df_plan_fc=sale_plan_fc,
    sale_plan_df=sale_plan_df,
)


NameError: name 'seasonality' is not defined

In [ ]:
###############################

In [7]:
# ============================================================
# FULL PIPELINE – LIFECYCLE + SALE PLAN + CALIB + SEASON + EXPORT
# ============================================================

# Nếu cần autoreload
%load_ext autoreload
%autoreload 2

# -----------------------------
# 0) IMPORT
# -----------------------------
import pandas as pd
import numpy as np

from src.config import CFG, BUCKETS_CANON

from src.rollrate.transition import compute_transition_by_mob
from src.rollrate.forecast import forecast_all_vintages
from src.rollrate.forecast_plan import forecast_sale_plan_by_mob

from src.rollrate.lifecycle import (
    get_actual_all_vintages_amount,
    combine_all_lifecycle_amount,
    lifecycle_to_long_df_amount,
    tag_forecast_rows_amount,
    add_del_metrics,
    aggregate_to_product,
    aggregate_products_to_portfolio,
    export_lifecycle_all_products_one_file,
    extend_actual_info_with_portfolio,
    BUCKETS_30P, BUCKETS_60P, BUCKETS_90P,
)

from src.rollrate.calibration import (
    compute_k_per_product_ifrs,
    apply_k_to_lifecycle,
    apply_k_to_sale_plan,
)

from src.rollrate.seasonality import (
    build_seasonality,
    apply_seasonality_to_lifecycle,
    apply_seasonality_to_sale_plan,
)

# -----------------------------
# 1) HELPER CHO SALE PLAN
# -----------------------------

def normalize_score(x):
    if pd.isna(x):
        return "ALL"
    s = str(x).strip().upper()
    if s in ["", "NA", "NAN", "NONE", "NULL"]:
        return "ALL"
    return s


def add_del_metrics_for_sale_plan(
    df_plan_fc: pd.DataFrame,
    sale_plan_df: pd.DataFrame,
) -> pd.DataFrame:
    """
    Tính DEL30/60/90 cho SALE PLAN forecast.

    - df_plan_fc: output từ forecast_sale_plan_by_mob
        cột bắt buộc:
            PRODUCT_TYPE, RISK_SCORE, VINTAGE_DATE, MOB
            các bucket EAD: DPD0, DPD1+, DPD30+, ..., WRITEOFF, PREPAY

    - sale_plan_df:
        cohort-level:
            PRODUCT_TYPE, RISK_SCORE, VINTAGE_DATE, EAD_PLAN (tổng disb / EAD gốc)
    """
    df = df_plan_fc.copy()

    sp = sale_plan_df.copy()
    sp["VINTAGE_DATE"] = pd.to_datetime(sp["VINTAGE_DATE"])

    # DISB_TOTAL = EAD_PLAN per cohort
    disb_map = (
        sp[["PRODUCT_TYPE", "RISK_SCORE", "VINTAGE_DATE", "EAD_PLAN"]]
        .rename(columns={"EAD_PLAN": "DISB_TOTAL"})
    )

    df = df.merge(
        disb_map,
        on=["PRODUCT_TYPE", "RISK_SCORE", "VINTAGE_DATE"],
        how="left",
    )

    # DEL amount
    df["DEL30_AMT"] = df[BUCKETS_30P].sum(axis=1)
    df["DEL60_AMT"] = df[BUCKETS_60P].sum(axis=1)
    df["DEL90_AMT"] = df[BUCKETS_90P].sum(axis=1)

    denom = df["DISB_TOTAL"].replace(0, np.nan)

    df["DEL30_PCT"] = df["DEL30_AMT"] / denom
    df["DEL60_PCT"] = df["DEL60_AMT"] / denom
    df["DEL90_PCT"] = df["DEL90_AMT"] / denom

    df["IS_FORECAST"] = 1
    df["SOURCE"] = "SALE_PLAN"

    return df


# ============================================================
# STEP 1 – RAW DATA & TRANSITION MATRICES
# ============================================================

# Giả sử df đã load trước đó, nếu không thì:
# df = pd.read_parquet("...")

df_raw = df.copy()
orig_col = CFG["orig_date"]
df_raw[orig_col] = pd.to_datetime(df_raw[orig_col])

print("✔ df_raw:", df_raw.shape)

matrices_by_mob, parent_fallback = compute_transition_by_mob(df_raw)
print("✔ matrices_by_mob built")

# ============================================================
# STEP 2 – FULL LIFECYCLE (ACTUAL + FORECAST, EAD)
# ============================================================

# 2.1 Actual EAD theo vintage
actual_results = get_actual_all_vintages_amount(df_raw)

# 2.2 Forecast EAD theo vintage
forecast_results = forecast_all_vintages(
    df_raw=df_raw,
    matrices_by_mob=matrices_by_mob,
    max_mob=29,
    enable_macro=False,
)

# 2.3 Merge actual + forecast vào lifecycle dict, rồi long format
lifecycle = combine_all_lifecycle_amount(actual_results, forecast_results)
df_lifecycle_ead = lifecycle_to_long_df_amount(lifecycle)

print("✔ df_lifecycle_ead:", df_lifecycle_ead.shape)

# Tag Forecast rows (IS_FORECAST)
df_lifecycle_tagged = tag_forecast_rows_amount(df_lifecycle_ead, df_raw)
print("✔ df_lifecycle_tagged:", df_lifecycle_tagged.shape)

# ============================================================
# STEP 3 – DEL METRICS (FOR CALIBRATION & SEASONALITY)
# ============================================================

df_lifecycle_metrics = add_del_metrics(df_lifecycle_tagged, df_raw)
print("✔ df_lifecycle_metrics (có DELxx_PCT):", df_lifecycle_metrics.shape)

# Tách phần forecast để dùng làm "full forecast" cho k
df_forecast_for_k = df_lifecycle_metrics[df_lifecycle_metrics["IS_FORECAST"] == 1].copy()

# ============================================================
# STEP 4 – CALIBRATION k PER PRODUCT (IFRS, TRIMMED MEAN)
# ============================================================

k_dict = compute_k_per_product_ifrs(
    df_actual_lifecycle=df_lifecycle_metrics,   # có cả actual & forecast, IS_FORECAST flag
    df_full_forecast=df_forecast_for_k,         # chỉ forecast rows
    default_anchor_mob=24,
    min_obs=3,
    method="trimmed_mean",
    trim_prop=0.2,
)

print("✔ k_dict:", k_dict)

# ============================================================
# STEP 5 – ÁP k VÀO LIFECYCLE (CHỈ FORECAST, KHÔNG ĐỤNG ACTUAL)
# ============================================================

df_lifecycle_calib_ead = apply_k_to_lifecycle(
    df_lifecycle=df_lifecycle_tagged,
    k_dict=k_dict,
    default_apply_mob=24,  # H_APPLY default cho product không có trong H_MAP_APPLY
)

print("✔ df_lifecycle_calib_ead:", df_lifecycle_calib_ead.shape)

# ============================================================
# STEP 6 – SEASONALITY & ÁP VÀO LIFECYCLE
# ============================================================

# Build seasonality từ actual DEL90 (không cần k, vì actual không đổi)
df_actual_for_season = df_lifecycle_metrics[df_lifecycle_metrics["IS_FORECAST"] == 0].copy()
seasonality = build_seasonality(df_actual_for_season, smooth_window=3)
print("✔ seasonality products:", list(seasonality.keys()))

# Apply seasonality lên EAD (đã calibrated)
df_lifecycle_season_ead = apply_seasonality_to_lifecycle(
    df_lifecycle=df_lifecycle_calib_ead,
    seasonality=seasonality,
)

print("✔ df_lifecycle_season_ead:", df_lifecycle_season_ead.shape)

# Sau khi đã apply k + seasonality trên EAD → tính lại DEL metrics CHUẨN
df_lifecycle_final = add_del_metrics(df_lifecycle_season_ead, df_raw)

# Bảo đảm flag
if "IS_FORECAST" not in df_lifecycle_final.columns:
    df_lifecycle_final = tag_forecast_rows_amount(df_lifecycle_final, df_raw)

df_lifecycle_final["SOURCE"] = df_lifecycle_final.get("SOURCE", "Historical")
df_lifecycle_final["SOURCE"] = df_lifecycle_final["SOURCE"].fillna("Historical")

print("✔ df_lifecycle_final:", df_lifecycle_final.shape)

# ============================================================
# STEP 7 – SALE PLAN: FORECAST + k + SEASONALITY + DEL
# ============================================================

# 7.1 Load sale plan
SALE_PLAN_PATH = r"C:/Users/MAFC4709/Python_work/RR_model/sale_plan_5yF2.xlsx"
SALE_PLAN_SHEET = "SALE_PLAN"

sale_plan_df = pd.read_excel(SALE_PLAN_PATH, sheet_name=SALE_PLAN_SHEET)
sale_plan_df["VINTAGE_DATE"] = pd.to_datetime(sale_plan_df["VINTAGE_DATE"])
sale_plan_df["RISK_SCORE"] = sale_plan_df["RISK_SCORE"].apply(normalize_score)

print("✔ sale_plan_df:", sale_plan_df.shape)

# 7.2 Forecast EAD cho sale plan
states = list(BUCKETS_CANON)

sale_plan_fc_ead = forecast_sale_plan_by_mob(
    sale_plan_df=sale_plan_df,
    matrices_by_mob=matrices_by_mob,
    parent_fallback=parent_fallback,
    mob_target=24,
    start_state=BUCKETS_CANON[0],
    states=states,
)

print("✔ sale_plan_fc_ead:", sale_plan_fc_ead.shape)

# 7.3 Áp k (tất cả MOB vì sale plan hoàn toàn là forecast)
sale_plan_fc_calib = apply_k_to_sale_plan(
    df_plan_fc=sale_plan_fc_ead,
    k_dict=k_dict,
)

# 7.4 Áp seasonality
sale_plan_fc_season_ead = apply_seasonality_to_sale_plan(
    df_plan_fc=sale_plan_fc_calib,
    seasonality=seasonality,
)

# 7.5 Tính DEL metrics cho sale plan (KHÔNG dùng add_del_metrics loan-level!)
sale_plan_fc_final = add_del_metrics_for_sale_plan(
    df_plan_fc=sale_plan_fc_season_ead,
    sale_plan_df=sale_plan_df,
)

print("✔ sale_plan_fc_final:", sale_plan_fc_final.shape)

# ============================================================
# STEP 8 – COMBINE LIFECYCLE + SALE PLAN → PRODUCT → PORTFOLIO → EXPORT
# ============================================================

# 8.1 GHÉP LIFECYCLE & SALE PLAN
df_all = pd.concat(
    [df_lifecycle_final, sale_plan_fc_final],
    ignore_index=True
).sort_values(
    ["PRODUCT_TYPE", "RISK_SCORE", "VINTAGE_DATE", "MOB"]
)

print("✔ df_all (lifecycle + sale plan):", df_all.shape)

# 8.2 AGGREGATE Product × Score → Product
df_prod = aggregate_to_product(df_all)
print("✔ df_prod:", df_prod.shape)

# 8.3 AGGREGATE Products → Portfolio
df_port = aggregate_products_to_portfolio(
    df_del_prod=df_prod,
    portfolio_name="PORTFOLIO_ALL",
)
print("✔ df_port:", df_port.shape)

df_prod_port = pd.concat([df_prod, df_port], ignore_index=True)

# 8.4 actual_info để vẽ boundary đỏ (max MOB actual theo Product × Vintage)
actual_info_prod = (
    df_all[df_all["IS_FORECAST"] == 0]
    .groupby(["PRODUCT_TYPE", "VINTAGE_DATE"])["MOB"]
    .max()
    .to_dict()
)

actual_info = extend_actual_info_with_portfolio(
    actual_info_prod,
    portfolio_name="PORTFOLIO_ALL",
)

print("✔ actual_info:", len(actual_info), "cohorts")

# 8.5 EXPORT EXCEL
output_file = "Lifecycle_All_Products_With_SalePlan_Calib_Season.xlsx"

export_lifecycle_all_products_one_file(
    df_del_prod=df_prod_port,
    actual_info=actual_info,
    filename=output_file,
)

print(f"🎉 DONE — Exported {output_file}")


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
✔ df_raw: (25998763, 14)
⚠️ Có 4 hàng có tổng weight = 0: ['DPD120+', 'DPD180+', 'PREPAY', 'SOLDOUT']
⚙️ Built parent fallback for (product=CDLPIL, score=500-)
⚠️ Có 4 hàng có tổng weight = 0: ['DPD120+', 'DPD180+', 'PREPAY', 'SOLDOUT']
⚙️ Built parent fallback for (product=CDLPIL, score=550-)
⚠️ Có 4 hàng có tổng weight = 0: ['DPD120+', 'DPD180+', 'PREPAY', 'SOLDOUT']
⚙️ Built parent fallback for (product=CDLPIL, score=600-)
⚠️ Có 4 hàng có tổng weight = 0: ['DPD120+', 'DPD180+', 'PREPAY', 'SOLDOUT']
⚙️ Built parent fallback for (product=CDLPIL, score=650+)
⚠️ Có 4 hàng có tổng weight = 0: ['DPD120+', 'DPD180+', 'PREPAY', 'SOLDOUT']
⚙️ Built parent fallback for (product=CDLPIL, score=650-)
⚠️ Có 4 hàng có tổng weight = 0: ['DPD120+', 'DPD180+', 'PREPAY', 'SOLDOUT']
⚙️ Built parent fallback for (product=CDLPIL, score=OTHERS)
⚠️ Có 4 hàng có tổng weight = 0: ['DPD120+', 'DPD180+', 'PREPAY', 'SOLDOUT'

C:\Users\MAFC4709\Python_work\RR_model\src\rollrate\lifecycle.py:416: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: pd.Series({


✔ df_prod: (14730, 7)


C:\Users\MAFC4709\Python_work\RR_model\src\rollrate\lifecycle.py:482: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(


✔ df_port: (2540, 7)
✔ actual_info: 214 cohorts
✔ Export lifecycle multi-product thành công → Lifecycle_All_Products_With_SalePlan_Calib_Season.xlsx
🎉 DONE — Exported Lifecycle_All_Products_With_SalePlan_Calib_Season.xlsx


In [8]:
print("✔ k_dict:", k_dict)


✔ k_dict: {'CDLPIL': 1.0, 'SALPIL': 1.0, 'SPLPIL': 1.0, 'TOPUP': 1.0, 'TWLPIL': 1.0, 'XSELL': 1.0}


In [9]:
print("✔ seasonality products:", list(seasonality.keys()))

✔ seasonality products: ['CDLPIL', 'SALPIL', 'SPLPIL', 'TOPUP', 'TWLPIL', 'XSELL']


In [10]:
pd.DataFrame(seasonality)


,CDLPIL,SALPIL,SPLPIL,TOPUP,TWLPIL,XSELL
MONTH,,,,,,
1,1.016873,1.088910,1.193736,1.111511,1.535029,1.044755
2,1.029604,1.062190,1.206691,1.135880,1.342283,1.069862
3,1.012890,1.053293,1.208758,1.112772,1.324292,1.097166
4,0.973830,1.022368,1.162532,1.065318,0.871737,1.099454
5,0.905429,0.968693,1.008770,0.980694,0.823216,1.034091
6,0.882128,0.910228,0.792883,0.935709,0.509291,0.979052
7,0.861602,0.834602,0.608137,0.884245,0.474688,0.884693
8,0.852631,0.826606,0.631723,0.844579,0.454787,0.827365
9,0.974947,0.940063,0.962767,0.895393,1.062617,0.867206


In [12]:
# ===============================================================
# 0) IMPORT TẤT CẢ NHỮNG GÌ CẦN DÙNG
# ===============================================================
import pandas as pd
import numpy as np

from src.rollrate.calibration import get_anchor_for_product
# df_lifecycle_metrics và df_forecast_for_k đã có sẵn từ pipeline trước đó


# ===============================================================
# 1) KIỂM TRA ACTUAL TẠI ANCHOR MOB
# ===============================================================

print("===== CHECK ACTUAL DEL90 AT ANCHOR MOB =====")

products = df_lifecycle_metrics["PRODUCT_TYPE"].unique()

for p in products:
    H = get_anchor_for_product(p)

    tmp_actual = df_lifecycle_metrics[
        (df_lifecycle_metrics["PRODUCT_TYPE"] == p) &
        (df_lifecycle_metrics["MOB"] == H) &
        (df_lifecycle_metrics["IS_FORECAST"] == 0)
    ][["PRODUCT_TYPE", "RISK_SCORE", "VINTAGE_DATE", "DEL90_PCT"]]

    print(f"\n### PRODUCT = {p}, ANCHOR = {H}")
    print("Actual sample:")
    print(tmp_actual.head())
    print("Describe actual:")
    print(tmp_actual["DEL90_PCT"].describe())


# ===============================================================
# 2) KIỂM TRA FORECAST TẠI ANCHOR MOB
# ===============================================================

print("\n===== CHECK FORECAST DEL90 AT ANCHOR MOB =====")

for p in products:
    H = get_anchor_for_product(p)

    tmp_fc = df_forecast_for_k[
        (df_forecast_for_k["PRODUCT_TYPE"] == p) &
        (df_forecast_for_k["MOB"] == H)
    ][["PRODUCT_TYPE", "RISK_SCORE", "VINTAGE_DATE", "DEL90_PCT"]]

    print(f"\n### PRODUCT = {p}, ANCHOR = {H}")
    print("Forecast sample:")
    print(tmp_fc.head())
    print("Describe forecast:")
    print(tmp_fc["DEL90_PCT"].describe())


# ===============================================================
# 3) MERGE ACTUAL vs FORECAST → XEM RATIO
# ===============================================================

print("\n===== MERGE ACTUAL vs FORECAST — RATIO =====")

merged_debug = df_lifecycle_metrics[
    df_lifecycle_metrics["IS_FORECAST"] == 0
][["PRODUCT_TYPE", "RISK_SCORE", "VINTAGE_DATE", "MOB", "DEL90_PCT"]].rename(
    columns={"DEL90_PCT": "DEL90_ACT"}
).merge(
    df_forecast_for_k[
        ["PRODUCT_TYPE", "RISK_SCORE", "VINTAGE_DATE", "MOB", "DEL90_PCT"]
    ].rename(columns={"DEL90_PCT": "DEL90_FC"}),
    on=["PRODUCT_TYPE", "RISK_SCORE", "VINTAGE_DATE", "MOB"],
    how="inner"
)

merged_debug["ratio"] = merged_debug["DEL90_ACT"] / merged_debug["DEL90_FC"].replace(0, np.nan)
merged_debug = merged_debug.replace([np.inf, -np.inf], np.nan)

print("\nSample merged rows:")
print(merged_debug.head())

print("\n===== RATIO SUMMARY BY PRODUCT =====")

for p in products:
    tmp = merged_debug[merged_debug["PRODUCT_TYPE"] == p]
    print(f"\n### PRODUCT {p}")
    print(tmp["ratio"].describe())


ImportError: cannot import name 'get_anchor_for_product' from 'src.rollrate.calibration' (C:\Users\MAFC4709\Python_work\RR_model\src\rollrate\calibration.py)

In [13]:
for p in df_actual_lifecycle["PRODUCT_TYPE"].unique():
    H = H_MAP_CALIB.get(p, 24)
    print("\nPRODUCT:", p, "ANCHOR:", H)

    print("Actual at H:")
    print(df_actual_lifecycle[
        (df_actual_lifecycle["PRODUCT_TYPE"] == p) &
        (df_actual_lifecycle["MOB"] == H) &
        (df_actual_lifecycle["IS_FORECAST"] == 0)
    ].head())


NameError: name 'df_actual_lifecycle' is not defined

In [14]:
# ===============================================================
# DEBUG k — XEM ACTUAL vs FORECAST TẠI ANCHOR MOB
# ===============================================================

import pandas as pd
import numpy as np

from src.rollrate.calibration import (
    H_MAP_CALIB,
    extract_actual_del90,
    extract_forecast_del90,
)

# Giả sử bạn đã chạy pipeline trước đó và đã có:
# - df_lifecycle_metrics  (lifecycle sau add_del_metrics, có DEL90_PCT, IS_FORECAST)
# - df_forecast_for_k     (phần forecast dùng để tính k, cũng có DEL90_PCT)

print("df_lifecycle_metrics shape:", df_lifecycle_metrics.shape)
print("df_forecast_for_k shape:", df_forecast_for_k.shape)

# ---------------------------------------------------------------
# 1) LẤY BẢNG ACTUAL & FORECAST DEL90 (DÙNG ĐÚNG HÀM TRONG calibration.py)
# ---------------------------------------------------------------

actual_all = extract_actual_del90(
    df_lifecycle=df_lifecycle_metrics,
    require_actual_flag=True,   # chỉ lấy IS_FORECAST == 0
)
forecast_all = extract_forecast_del90(
    df_full_forecast=df_forecast_for_k,
)

print("\nActual_all (head):")
print(actual_all.head())
print("\nForecast_all (head):")
print(forecast_all.head())

# ---------------------------------------------------------------
# 2) XEM ACTUAL & FORECAST TẠI MOB ANCHOR CHO TỪNG PRODUCT
# ---------------------------------------------------------------

products = sorted(actual_all["PRODUCT_TYPE"].unique())

print("\n================= CHECK TẠI MOB ANCHOR =================")

for p in products:
    H = H_MAP_CALIB.get(p, 24)

    act_p = actual_all[
        (actual_all["PRODUCT_TYPE"] == p) &
        (actual_all["MOB"] == H)
    ].copy()

    fc_p = forecast_all[
        (forecast_all["PRODUCT_TYPE"] == p) &
        (forecast_all["MOB"] == H)
    ].copy()

    print(f"\n>>> PRODUCT = {p}, ANCHOR MOB = {H}")
    print("Actual at H (sample):")
    print(act_p.head())

    print("Forecast at H (sample):")
    print(fc_p.head())

    print("Describe ACTUAL DEL90_ACT at H:")
    print(act_p["DEL90_ACT"].describe())

    print("Describe FORECAST DEL90_FC at H:")
    print(fc_p["DEL90_FC"].describe())

# ---------------------------------------------------------------
# 3) MERGE ACTUAL vs FORECAST TẠI MỌI MOB → XEM RATIO
# ---------------------------------------------------------------

merged_debug = actual_all.merge(
    forecast_all,
    on=["PRODUCT_TYPE", "RISK_SCORE", "VINTAGE_DATE", "MOB"],
    how="inner",
)

merged_debug["RATIO"] = (
    merged_debug["DEL90_ACT"] /
    merged_debug["DEL90_FC"].replace(0, np.nan)
)
merged_debug = merged_debug.replace([np.inf, -np.inf], np.nan)

print("\n================= MERGED SAMPLE =================")
print(merged_debug.head())

print("\n================= RATIO SUMMARY BY PRODUCT =================")
for p in sorted(merged_debug["PRODUCT_TYPE"].unique()):
    tmp = merged_debug[merged_debug["PRODUCT_TYPE"] == p]
    print(f"\n### PRODUCT {p}")
    print(tmp["RATIO"].describe())


df_lifecycle_metrics shape: (19170, 23)
df_forecast_for_k shape: (9159, 23)

Actual_all (head):
  PRODUCT_TYPE RISK_SCORE VINTAGE_DATE  MOB  DEL90_ACT
0       CDLPIL       500-   2023-01-01    0   0.000000
1       CDLPIL       500-   2023-01-01    1   0.000000
2       CDLPIL       500-   2023-01-01    2   0.000000
3       CDLPIL       500-   2023-01-01    3   0.000000
4       CDLPIL       500-   2023-01-01    4   0.035149

Forecast_all (head):
  PRODUCT_TYPE RISK_SCORE VINTAGE_DATE  MOB  DEL90_FC
0       CDLPIL       500-   2023-01-01   25  0.162762
1       CDLPIL       500-   2023-01-01   26  0.162762
2       CDLPIL       500-   2023-01-01   27  0.162762
3       CDLPIL       500-   2023-01-01   28  0.162762
4       CDLPIL       500-   2023-01-01   29  0.162762

================= CHECK TẠI MOB ANCHOR =================

>>> PRODUCT = CDLPIL, ANCHOR MOB = 12
Actual at H (sample):
    PRODUCT_TYPE RISK_SCORE VINTAGE_DATE  MOB  DEL90_ACT
12        CDLPIL       500-   2023-01-01   12   0.16

In [15]:
df_forecast_full = forecast_all_vintages(
    df_raw=df,
    matrices_by_mob=matrices_by_mob,
    max_mob=36,
    enable_macro=False
)

df_forecast_full_long = forecast_to_long_df(df_forecast_full)
df_forecast_full_long = add_del_metrics_simple(df_forecast_full_long)

# check forecast MOB12
df_forecast_full_long[df_forecast_full_long["MOB"] == 12]["DEL90_PCT"].describe()


NameError: name 'forecast_to_long_df' is not defined

In [16]:
import pandas as pd

from src.config import BUCKETS_CANON

def forecast_to_long_df(forecast_results: dict) -> pd.DataFrame:
    """
    Convert forecast_all_vintages() → long DataFrame:
        PRODUCT, SCORE, VINTAGE_DATE, MOB, BUCKET columns...
    """
    rows = []

    for (product, score, vintage), mob_dict in forecast_results.items():
        for mob, series in mob_dict.items():
            row = {
                "PRODUCT_TYPE": product,
                "RISK_SCORE": score,
                "VINTAGE_DATE": vintage,
                "MOB": mob
            }
            for bucket in BUCKETS_CANON:
                row[bucket] = series.get(bucket, 0.0)
            rows.append(row)

    df = pd.DataFrame(rows)

    # Ensure correct type
    df["VINTAGE_DATE"] = pd.to_datetime(df["VINTAGE_DATE"])
    df["MOB"] = df["MOB"].astype(int)

    return df.sort_values(["PRODUCT_TYPE", "RISK_SCORE", "VINTAGE_DATE", "MOB"])


In [17]:
from src.rollrate.lifecycle import BUCKETS_30P, BUCKETS_60P, BUCKETS_90P

def add_del_metrics_simple(df_lifecycle: pd.DataFrame) -> pd.DataFrame:
    """
    Rất quan trọng: dùng cho forecast_long_df.
    Không có DISB_TOTAL từ df_raw, nên nếu VINTAGE trùng nhau
    → bạn cần có cột DISB_TOTAL trước đó.

    Nếu chưa có DISB_TOTAL → mặc định lấy tổng EAD tại MOB0.
    """

    df = df_lifecycle.copy()

    # Nếu chưa có DISB_TOTAL → tạo từ MOB0
    if "DISB_TOTAL" not in df.columns:
        df0 = (
            df[df["MOB"] == 0]
            .groupby(["PRODUCT_TYPE", "RISK_SCORE", "VINTAGE_DATE"])[BUCKETS_CANON[0]]
            .sum()
            .rename("DISB_TOTAL")
        )

        df = df.merge(
            df0,
            on=["PRODUCT_TYPE", "RISK_SCORE", "VINTAGE_DATE"],
            how="left"
        )

    # Tính AMT
    df["DEL30_AMT"] = df[BUCKETS_30P].sum(axis=1)
    df["DEL60_AMT"] = df[BUCKETS_60P].sum(axis=1)
    df["DEL90_AMT"] = df[BUCKETS_90P].sum(axis=1)

    # Tính tỷ lệ
    df["DEL30_PCT"] = df["DEL30_AMT"] / df["DISB_TOTAL"]
    df["DEL60_PCT"] = df["DEL60_AMT"] / df["DISB_TOTAL"]
    df["DEL90_PCT"] = df["DEL90_AMT"] / df["DISB_TOTAL"]

    return df


In [18]:
from src.rollrate.forecast import forecast_all_vintages
from src.rollrate.lifecycle import BUCKETS_30P, BUCKETS_60P, BUCKETS_90P

# 1. Forecast từ MOB0 → max_mob cho toàn bộ lịch sử
df_forecast_full = forecast_all_vintages(
    df_raw=df,
    matrices_by_mob=matrices_by_mob,
    max_mob=36,
    enable_macro=False
)
print("✔ Full forecast computed.")

# 2. Convert về long format
df_forecast_full_long = forecast_to_long_df(df_forecast_full)
print("✔ Converted to long format:", df_forecast_full_long.shape)

# 3. Tính DEL metrics cho full forecast
df_forecast_full_long = add_del_metrics_simple(df_forecast_full_long)
print("✔ Added DEL metrics:", df_forecast_full_long.shape)

# 4. Kiểm tra forecast tại MOB12
print("\n=== Forecast DEL90_PCT tại MOB12 ===")
print(df_forecast_full_long[df_forecast_full_long["MOB"] == 12]["DEL90_PCT"].describe())


✔ Full forecast computed.
✔ Converted to long format: (14271, 14)
✔ Added DEL metrics: (14271, 21)

=== Forecast DEL90_PCT tại MOB12 ===
count    20.000000
mean      0.178235
std       0.109492
min       0.037214
25%       0.112362
50%       0.148538
75%       0.213075
max       0.495496
Name: DEL90_PCT, dtype: float64


In [19]:
from src.rollrate.calibration import *
from src.rollrate.lifecycle import add_del_metrics

# Compute actual lifecycle metrics
df_lifecycle_metrics = add_del_metrics(df_lifecycle, df_raw=df)
df_lifecycle_metrics["IS_FORECAST"] = 0

# Compute full forecast metrics
df_forecast_full_long = add_del_metrics_simple(df_forecast_full_long)
df_forecast_full_long["IS_FORECAST"] = 1

# Tính k
k_dict = compute_k_per_product_ifrs(
    df_actual_lifecycle=df_lifecycle_metrics,
    df_full_forecast=df_forecast_full_long,
)

print("\n================= K DICT =================")
print(k_dict)


NameError: name 'df_lifecycle' is not defined

In [ ]:
from src.rollrate.calibration import H_MAP_CALIB
from src.rollrate.calibration import extract_actual_del90, extract_forecast_del90

df_actual_anchor = extract_actual_del90(df_lifecycle_metrics)
df_forecast_anchor = extract_forecast_del90(df_forecast_full_long)

print("\n================= CHECK TẠI ANCHOR MOB =================")

for p in sorted(df_actual_anchor["PRODUCT_TYPE"].unique()):
    H = H_MAP_CALIB.get(p, 24)
    print(f"\n>>> PRODUCT = {p}, ANCHOR MOB = {H}")

    act = df_actual_anchor[
        (df_actual_anchor["PRODUCT_TYPE"] == p) &
        (df_actual_anchor["MOB"] == H)
    ]

    fc = df_forecast_anchor[
        (df_forecast_anchor["PRODUCT_TYPE"] == p) &
        (df_forecast_anchor["MOB"] == H)
    ]

    print("Actual sample:\n", act.head())
    print("Forecast sample:\n", fc.head())

    print("Actual describe:\n", act["DEL90_ACT"].describe())
    print("Forecast describe:\n", fc["DEL90_FC"].describe())


In [21]:
def add_del_metrics_simple(df_lifecycle: pd.DataFrame) -> pd.DataFrame:
    """
    Rất quan trọng: dùng cho forecast_long_df.
    Không có DISB_TOTAL từ df_raw, nên nếu VINTAGE trùng nhau
    → bạn cần có cột DISB_TOTAL trước đó.

    Nếu chưa có DISB_TOTAL → mặc định lấy tổng EAD tại MOB0.
    """

    df = df_lifecycle.copy()

    # Nếu chưa có DISB_TOTAL → tạo từ MOB0
    if "DISB_TOTAL" not in df.columns:
        df0 = (
            df[df["MOB"] == 0]
            .groupby(["PRODUCT_TYPE", "RISK_SCORE", "VINTAGE_DATE"])[BUCKETS_CANON[0]]
            .sum()
            .rename("DISB_TOTAL")
        )

        df = df.merge(
            df0,
            on=["PRODUCT_TYPE", "RISK_SCORE", "VINTAGE_DATE"],
            how="left"
        )

    # Tính AMT
    df["DEL30_AMT"] = df[BUCKETS_30P].sum(axis=1)
    df["DEL60_AMT"] = df[BUCKETS_60P].sum(axis=1)
    df["DEL90_AMT"] = df[BUCKETS_90P].sum(axis=1)

    # Tính tỷ lệ
    df["DEL30_PCT"] = df["DEL30_AMT"] / df["DISB_TOTAL"]
    df["DEL60_PCT"] = df["DEL60_AMT"] / df["DISB_TOTAL"]
    df["DEL90_PCT"] = df["DEL90_AMT"] / df["DISB_TOTAL"]

    return df

In [22]:
# =============================================================
# 📌 1) IMPORT
# =============================================================
import pandas as pd
import numpy as np

from src.rollrate.calibration import (
    compute_k_per_product_ifrs,
    extract_actual_del90,
    extract_forecast_del90,
)

from src.rollrate.forecast import forecast_all_vintages


# =============================================================
# 📌 2) BẢO ĐẢM lifecycle đã có (actual + forecast)
#     → df_lifecycle_metrics phải tồn tại trước đó
# =============================================================
try:
    df_lifecycle_metrics
except NameError:
    raise RuntimeError("❌ df_lifecycle_metrics chưa có — hãy chạy pipeline lifecycle trước!")

print("✔ df_lifecycle_metrics available:", df_lifecycle_metrics.shape)


# =============================================================
# 📌 3) BUILD FULL-HISTORY FORECAST (MOB 0 → max)
#     → dùng để so sánh với actual tại MOB anchor
# =============================================================
print("\n[Build full forecast from raw data]")

# full forecast cho historical (chỉ từ df_raw, không có sale plan)
df_forecast_full = forecast_all_vintages(
    df_raw=df,
    matrices_by_mob=matrices_by_mob,
    max_mob=36,
    enable_macro=False
)

# Convert dict → long DF
from src.rollrate.lifecycle import lifecycle_to_long_df_amount

df_forecast_full_long = lifecycle_to_long_df_amount(df_forecast_full)
df_forecast_full_long = add_del_metrics_simple(df_forecast_full_long)

print("✔ df_forecast_full_long:", df_forecast_full_long.shape)


# =============================================================
# 📌 4) LẤY DEL90 ACTUAL vs FORECAST TẠI MỌB ANCHOR
# =============================================================
print("\n[Extract actual + forecast DEL90 full-history]")

df_actual_H = extract_actual_del90(df_lifecycle_metrics)
df_fc_H = extract_forecast_del90(df_forecast_full_long)

print("✔ df_actual_H:", df_actual_H.shape)
print("✔ df_fc_H:", df_fc_H.shape)


# =============================================================
# 📌 5) KIỂM TRA THEO PRODUCT (đây quan trọng nhất)
# =============================================================

from src.rollrate.calibration import H_MAP_CALIB

products = sorted(df_actual_H["PRODUCT_TYPE"].unique())
print("\n==================== CALIB CHECK PER PRODUCT ====================\n")

for p in products:
    H = H_MAP_CALIB.get(p, 24)

    print(f"===== PRODUCT = {p} | ANCHOR MOB = {H} =====")

    # Actual tại anchor
    actual_p = df_actual_H[
        (df_actual_H["PRODUCT_TYPE"] == p) &
        (df_actual_H["MOB"] == H)
    ]

    # Forecast tại anchor
    forecast_p = df_fc_H[
        (df_fc_H["PRODUCT_TYPE"] == p) &
        (df_fc_H["MOB"] == H)
    ]

    print(f"Actual count at H:   {actual_p.shape[0]}")
    print(f"Forecast count at H: {forecast_p.shape[0]}")

    if actual_p.empty or forecast_p.empty:
        print("⚠️ -> Skip (không có sample để tính k)\n")
        continue

    print("\n--- Actual (sample) ---")
    print(actual_p.head())

    print("\n--- Forecast (sample) ---")
    print(forecast_p.head())

    print("\n--- Describe ACTUAL DEL90 at H ---")
    print(actual_p["DEL90_ACT"].describe())

    print("\n--- Describe FORECAST DEL90 at H ---")
    print(forecast_p["DEL90_FC"].describe())

    # Merge để tính ratio
    merged = actual_p.merge(
        forecast_p,
        on=["PRODUCT_TYPE", "RISK_SCORE", "VINTAGE_DATE", "MOB"],
        how="inner",
    )

    merged["RATIO"] = merged["DEL90_ACT"] / merged["DEL90_FC"].replace(0, np.nan)
    merged = merged.replace([np.inf, -np.inf], np.nan).dropna()

    print("\n--- Ratio ACTUAL/FORECAST ---")
    print(merged["RATIO"].describe())

    print("\n--------------------------------------------------------\n")


# =============================================================
# 📌 6) TÍNH k (FINAL)
# =============================================================
print("\n==================== FINAL k PER PRODUCT ====================\n")

k_dict = compute_k_per_product_ifrs(
    df_actual_lifecycle=df_lifecycle_metrics,
    df_full_forecast=df_forecast_full_long,
    default_anchor_mob=24,
    min_obs=5,
    method="trimmed_mean",
    trim_prop=0.2,
)

print("k_dict =", k_dict)


✔ df_lifecycle_metrics available: (19170, 23)

[Build full forecast from raw data]
✔ df_forecast_full_long: (14271, 21)

[Extract actual + forecast DEL90 full-history]
✔ df_actual_H: (10011, 5)
✔ df_fc_H: (14271, 5)

==================== CALIB CHECK PER PRODUCT ====================

===== PRODUCT = CDLPIL | ANCHOR MOB = 12 =====
Actual count at H:   122
Forecast count at H: 78

--- Actual (sample) ---
    PRODUCT_TYPE RISK_SCORE VINTAGE_DATE  MOB  DEL90_ACT
12        CDLPIL       500-   2023-01-01   12   0.167500
37        CDLPIL       500-   2023-02-01   12   0.254235
62        CDLPIL       500-   2023-03-01   12   0.259826
87        CDLPIL       500-   2023-04-01   12   0.264098
112       CDLPIL       500-   2023-05-01   12   0.312721

--- Forecast (sample) ---
    PRODUCT_TYPE RISK_SCORE VINTAGE_DATE  MOB  DEL90_FC
326       CDLPIL       500-   2024-09-01   12       NaN
352       CDLPIL       500-   2024-10-01   12       NaN
379       CDLPIL       500-   2024-11-01   12       NaN
40

In [23]:
import pandas as pd
import numpy as np

from src.config import CFG
from src.rollrate.forecast import forecast_all_vintages
from src.rollrate.lifecycle import (
    lifecycle_to_long_df_amount,
    add_del_metrics,
)
from src.rollrate.calibration import (
    compute_k_per_product_ifrs,
    extract_actual_del90,
    extract_forecast_del90,
    H_MAP_CALIB,
)

# =========================================================
# 1) LẤY TẬP COHORT THỰC TẾ (WINDOW CHUẨN CHO k)
#    → chỉ lấy các cohort có ACTUAL (IS_FORECAST == 0)
# =========================================================

orig_col = CFG["orig_date"]  # ví dụ: "DISBURSAL_DATE"

if "IS_FORECAST" not in df_lifecycle_metrics.columns:
    raise KeyError("df_lifecycle_metrics cần có IS_FORECAST để xác định actual vs forecast.")

actual_cohorts = (
    df_lifecycle_metrics[df_lifecycle_metrics["IS_FORECAST"] == 0]
    [["PRODUCT_TYPE", "RISK_SCORE", "VINTAGE_DATE"]]
    .drop_duplicates()
)

print("✔ Số cohort actual dùng làm window calibration:", actual_cohorts.shape[0])

# Map về raw df bằng orig_col
actual_cohorts_raw_key = actual_cohorts.rename(columns={"VINTAGE_DATE": orig_col})

# =========================================================
# 2) FILTER df_raw CHỈ CÒN CÁC COHORT CÓ ACTUAL
#    → df_hist là historical window dùng để build forecast cho k
# =========================================================

df_hist = df.merge(
    actual_cohorts_raw_key,
    on=["PRODUCT_TYPE", "RISK_SCORE", orig_col],
    how="inner"
)

print("✔ df_hist shape (raw trong window calibration):", df_hist.shape)

# =========================================================
# 3) FORECAST FULL-HISTORY TRÊN CHÍNH WINDOW ĐÓ
#    → đảm bảo forecast_vintages == actual_vintages
# =========================================================

forecast_hist = forecast_all_vintages(
    df_raw=df_hist,
    matrices_by_mob=matrices_by_mob,
    max_mob=36,
    enable_macro=False,
)

print("✔ forecast_hist segments:", len(forecast_hist))

# Convert dict → long format
df_forecast_hist_long = lifecycle_to_long_df_amount(forecast_hist)

# Thêm DEL30/60/90 + DISB_TOTAL dựa trên CHÍNH df_hist
df_forecast_hist_long = add_del_metrics(df_forecast_hist_long, df_raw=df_hist)

print("✔ df_forecast_hist_long shape:", df_forecast_hist_long.shape)

# =========================================================
# 4) DEBUG: CHECK COHORT GIAO NHAU TẠI MOB ANCHOR (VD: CDLPIL)
# =========================================================

df_actual_H = extract_actual_del90(df_lifecycle_metrics)
df_fc_H = extract_forecast_del90(df_forecast_hist_long)

print("\n[DEBUG] Check intersection tại anchor MOB cho từng product")

products = sorted(df_actual_H["PRODUCT_TYPE"].unique())
for p in products:
    H = H_MAP_CALIB.get(p, 24)
    act_p = df_actual_H[(df_actual_H["PRODUCT_TYPE"] == p) & (df_actual_H["MOB"] == H)]
    fc_p = df_fc_H[(df_fc_H["PRODUCT_TYPE"] == p) & (df_fc_H["MOB"] == H)]

    common = act_p.merge(
        fc_p,
        on=["PRODUCT_TYPE", "RISK_SCORE", "VINTAGE_DATE", "MOB"],
        how="inner",
    )

    print(f"\n>>> PRODUCT = {p}, ANCHOR MOB = {H}")
    print("  #actual at H:", act_p.shape[0])
    print("  #forecast at H:", fc_p.shape[0])
    print("  #intersection:", common.shape[0])

# =========================================================
# 5) TÍNH k PER PRODUCT TRÊN CÙNG WINDOW
# =========================================================

k_dict = compute_k_per_product_ifrs(
    df_actual_lifecycle=df_lifecycle_metrics,
    df_full_forecast=df_forecast_hist_long,
    default_anchor_mob=24,
    min_obs=5,
    method="trimmed_mean",  # hoặc "median"
    trim_prop=0.2,
)

print("\n================= k per product (calibrated trên cùng window) =================")
print(k_dict)


✔ Số cohort actual dùng làm window calibration: 639
✔ df_hist shape (raw trong window calibration): (25998763, 14)
✔ forecast_hist segments: 639
✔ df_forecast_hist_long shape: (14271, 21)

[DEBUG] Check intersection tại anchor MOB cho từng product

>>> PRODUCT = CDLPIL, ANCHOR MOB = 12
  #actual at H: 122
  #forecast at H: 78
  #intersection: 6

>>> PRODUCT = SALPIL, ANCHOR MOB = 24
  #actual at H: 36
  #forecast at H: 132
  #intersection: 36

>>> PRODUCT = SPLPIL, ANCHOR MOB = 12
  #actual at H: 21
  #forecast at H: 13
  #intersection: 1

>>> PRODUCT = TOPUP, ANCHOR MOB = 24
  #actual at H: 36
  #forecast at H: 132
  #intersection: 36

>>> PRODUCT = TWLPIL, ANCHOR MOB = 12
  #actual at H: 4
  #forecast at H: 13
  #intersection: 1

>>> PRODUCT = XSELL, ANCHOR MOB = 24
  #actual at H: 36
  #forecast at H: 132
  #intersection: 36

================= k per product (calibrated trên cùng window) =================
{'CDLPIL': 1.0, 'SALPIL': 1.0, 'SPLPIL': 1.0, 'TOPUP': 1.0, 'TWLPIL': 1.0, 'XSE

In [24]:
import pandas as pd
import numpy as np

from src.rollrate.calibration import (
    extract_actual_del90,
    extract_forecast_del90,
    H_MAP_CALIB,
)

# 1) Lấy bảng DEL90 theo MOB đầy đủ (không filter MOB H)
df_actual_all = extract_actual_del90(df_lifecycle_metrics)          # có DEL90_ACT
df_fc_all     = extract_forecast_del90(df_forecast_hist_long)       # có DEL90_FC

print("df_actual_all:", df_actual_all.shape)
print("df_fc_all    :", df_fc_all.shape)

# 2) So sánh key full 4 cột
keys_act = df_actual_all[["PRODUCT_TYPE","RISK_SCORE","VINTAGE_DATE","MOB"]].drop_duplicates()
keys_fc  = df_fc_all[["PRODUCT_TYPE","RISK_SCORE","VINTAGE_DATE","MOB"]].drop_duplicates()

keys_merge = keys_act.merge(
    keys_fc,
    on=["PRODUCT_TYPE","RISK_SCORE","VINTAGE_DATE","MOB"],
    how="outer",
    indicator=True
)

print("\n=== Tổng quan key alignment (all products) ===")
print(keys_merge["_merge"].value_counts())

# 3) Kiểm tra riêng 1 product, ví dụ CDLPIL
p = "CDLPIL"
keys_p = keys_merge[keys_merge["PRODUCT_TYPE"] == p]

print(f"\n=== Key alignment cho PRODUCT = {p} ===")
print(keys_p["_merge"].value_counts())
print("\nSample left_only (chỉ có ở actual):")
print(keys_p[keys_p["_merge"] == "left_only"].head(10))
print("\nSample right_only (chỉ có ở forecast):")
print(keys_p[keys_p["_merge"] == "right_only"].head(10))


df_actual_all: (10011, 5)
df_fc_all    : (14271, 5)

=== Tổng quan key alignment (all products) ===
_merge
right_only    13632
left_only      9372
both            639
Name: count, dtype: int64

=== Key alignment cho PRODUCT = CDLPIL ===
_merge
right_only    4128
left_only     2856
both           194
Name: count, dtype: int64

Sample left_only (chỉ có ở actual):
  PRODUCT_TYPE RISK_SCORE VINTAGE_DATE  MOB     _merge
0       CDLPIL       500-   2023-01-01    0  left_only
1       CDLPIL       500-   2023-01-01    1  left_only
2       CDLPIL       500-   2023-01-01    2  left_only
3       CDLPIL       500-   2023-01-01    3  left_only
4       CDLPIL       500-   2023-01-01    4  left_only
5       CDLPIL       500-   2023-01-01    5  left_only
6       CDLPIL       500-   2023-01-01    6  left_only
7       CDLPIL       500-   2023-01-01    7  left_only
8       CDLPIL       500-   2023-01-01    8  left_only
9       CDLPIL       500-   2023-01-01    9  left_only

Sample right_only (chỉ có ở fo

In [25]:
from src.rollrate.calibration import (
    trimmed_mean,
    H_MAP_CALIB,
)

# Lấy lại anchor-level DEL90 (đã có từ block debug trước)
df_actual_H = extract_actual_del90(df_lifecycle_metrics)
df_fc_H     = extract_forecast_del90(df_forecast_hist_long)

products = sorted(df_actual_H["PRODUCT_TYPE"].unique())

print("=========== KIỂM TRA RATIO ACTUAL / FORECAST TẠI MOB ANCHOR ===========\n")

for p in products:
    H = H_MAP_CALIB.get(p, 24)

    act_p = df_actual_H[
        (df_actual_H["PRODUCT_TYPE"] == p) &
        (df_actual_H["MOB"] == H)
    ].copy()

    fc_p = df_fc_H[
        (df_fc_H["PRODUCT_TYPE"] == p) &
        (df_fc_H["MOB"] == H)
    ].copy()

    merged = act_p.merge(
        fc_p,
        on=["PRODUCT_TYPE","RISK_SCORE","VINTAGE_DATE","MOB"],
        how="inner",
    )

    print(f"\n>>> PRODUCT = {p}, ANCHOR MOB = {H}")
    print("  #actual at H:", act_p.shape[0])
    print("  #forecast at H:", fc_p.shape[0])
    print("  #intersection:", merged.shape[0])

    if merged.empty:
        print("  ⚠️ Không có sample để tính ratio.")
        continue

    merged["RATIO"] = merged["DEL90_ACT"] / merged["DEL90_FC"].replace(0, np.nan)
    merged["RATIO"] = merged["RATIO"].replace([np.inf, -np.inf], np.nan)

    merged = merged.dropna(subset=["RATIO"])

    print("\n  RATIO describe:")
    print(merged["RATIO"].describe())

    # Median & trimmed_mean
    median_ratio = float(np.median(merged["RATIO"].values))
    tm_ratio     = trimmed_mean(merged["RATIO"].values, trim_prop=0.2)

    print(f"\n  Median(RATIO)      = {median_ratio:.4f}")
    print(f"  Trimmed_mean(RATIO)= {tm_ratio:.4f}")

    # Tỉ lệ quan sát có RATIO rất gần 1
    near_1 = np.mean(np.abs(merged["RATIO"] - 1.0) < 0.02)  # trong khoảng ±2%
    print(f"  % RATIO trong khoảng [0.98, 1.02]: {near_1*100:.1f}%")

    # In một vài dòng lệch hẳn để xem có outlier không
    print("\n  Sample RATIO xa 1 (> ±10%):")
    print(
        merged[ (merged["RATIO"] < 0.9) | (merged["RATIO"] > 1.1) ]
        [["RISK_SCORE","VINTAGE_DATE","DEL90_ACT","DEL90_FC","RATIO"]]
        .head(10)
    )
    print("\n" + "-"*70)


=========== KIỂM TRA RATIO ACTUAL / FORECAST TẠI MOB ANCHOR ===========


>>> PRODUCT = CDLPIL, ANCHOR MOB = 12
  #actual at H: 122
  #forecast at H: 78
  #intersection: 6

  RATIO describe:
count    6.0
mean     1.0
std      0.0
min      1.0
25%      1.0
50%      1.0
75%      1.0
max      1.0
Name: RATIO, dtype: float64

  Median(RATIO)      = 1.0000
  Trimmed_mean(RATIO)= 1.0000
  % RATIO trong khoảng [0.98, 1.02]: 100.0%

  Sample RATIO xa 1 (> ±10%):
Empty DataFrame
Columns: [RISK_SCORE, VINTAGE_DATE, DEL90_ACT, DEL90_FC, RATIO]
Index: []

----------------------------------------------------------------------

>>> PRODUCT = SALPIL, ANCHOR MOB = 24
  #actual at H: 36
  #forecast at H: 132
  #intersection: 36

  RATIO describe:
count    36.0
mean      1.0
std       0.0
min       1.0
25%       1.0
50%       1.0
75%       1.0
max       1.0
Name: RATIO, dtype: float64

  Median(RATIO)      = 1.0000
  Trimmed_mean(RATIO)= 1.0000
  % RATIO trong khoảng [0.98, 1.02]: 100.0%

  Sample RATIO

In [26]:
import pandas as pd
import numpy as np

from src.rollrate.calibration import (
    extract_actual_del90,
    extract_forecast_del90,
    H_MAP_CALIB,
    trimmed_mean,
)

# 1) Lấy full bảng DEL90 (theo mọi MOB)
df_actual_all = extract_actual_del90(df_lifecycle_metrics)
df_fc_all     = extract_forecast_del90(df_forecast_hist_long)

print("df_actual_all:", df_actual_all.shape)
print("df_fc_all    :", df_fc_all.shape)

# 2) Chỉ check PRODUCT = CDLPIL
p = "CDLPIL"
H = H_MAP_CALIB.get(p, 24)
print(f"\n>>> CHECK PRODUCT = {p}, ANCHOR MOB = {H}")

# Actual tại đúng MOB = H
act_H = (
    df_actual_all[
        (df_actual_all["PRODUCT_TYPE"] == p) &
        (df_actual_all["MOB"] == H)
    ]
    .sort_values(["RISK_SCORE", "VINTAGE_DATE"])
    .reset_index(drop=True)
)

# Forecast full-history tại đúng MOB = H
fc_H = (
    df_fc_all[
        (df_fc_all["PRODUCT_TYPE"] == p) &
        (df_fc_all["MOB"] == H)
    ]
    .sort_values(["RISK_SCORE", "VINTAGE_DATE"])
    .reset_index(drop=True)
)

print("\n--- Actual @H (CDLPIL) ---")
print(act_H.head(10))
print("... total rows:", act_H.shape[0])

print("\n--- Forecast @H (CDLPIL) ---")
print(fc_H.head(10))
print("... total rows:", fc_H.shape[0])

# 3) Join theo đúng key (PRODUCT, SCORE, VINTAGE, MOB)
merged_H = act_H.merge(
    fc_H,
    on=["PRODUCT_TYPE", "RISK_SCORE", "VINTAGE_DATE", "MOB"],
    how="inner",
    suffixes=("_ACT", "_FC")
)

print("\n--- INTERSECTION @H (CDLPIL) ---")
print("intersection rows:", merged_H.shape[0])
print(merged_H.head(10))

# 4) TÍNH RATIO = ACT / FORECAST
merged_H["RATIO"] = merged_H["DEL90_ACT"] / merged_H["DEL90_FC"].replace(0, np.nan)
merged_H["RATIO"] = merged_H["RATIO"].replace([np.inf, -np.inf], np.nan)
merged_H = merged_H.dropna(subset=["RATIO"])

print("\n--- RATIO ACTUAL / FORECAST @H ---")
print(merged_H[["RISK_SCORE", "VINTAGE_DATE", "DEL90_ACT", "DEL90_FC", "RATIO"]].head(20))

print("\nDescribe RATIO @H:")
print(merged_H["RATIO"].describe())

if not merged_H.empty:
    median_ratio = float(np.median(merged_H["RATIO"].values))
    tm_ratio     = trimmed_mean(merged_H["RATIO"].values, trim_prop=0.2)
    print(f"\nMedian(RATIO)        = {median_ratio:.4f}")
    print(f"Trimmed mean (20%)   = {tm_ratio:.4f}")

    near_1 = np.mean(np.abs(merged_H["RATIO"] - 1.0) < 0.02)  # ±2%
    print(f"% RATIO trong [0.98,1.02] = {near_1*100:.1f}%")
else:
    print("\n⚠️ Không có RATIO nào (intersection rỗng).")


df_actual_all: (10011, 5)
df_fc_all    : (14271, 5)

>>> CHECK PRODUCT = CDLPIL, ANCHOR MOB = 12

--- Actual @H (CDLPIL) ---
  PRODUCT_TYPE RISK_SCORE VINTAGE_DATE  MOB  DEL90_ACT
0       CDLPIL       500-   2023-01-01   12   0.167500
1       CDLPIL       500-   2023-02-01   12   0.254235
2       CDLPIL       500-   2023-03-01   12   0.259826
3       CDLPIL       500-   2023-04-01   12   0.264098
4       CDLPIL       500-   2023-05-01   12   0.312721
5       CDLPIL       500-   2023-06-01   12   0.304399
6       CDLPIL       500-   2023-07-01   12   0.327091
7       CDLPIL       500-   2023-08-01   12   0.303799
8       CDLPIL       500-   2023-09-01   12   0.289615
9       CDLPIL       500-   2023-10-01   12   0.292056
... total rows: 122

--- Forecast @H (CDLPIL) ---
  PRODUCT_TYPE RISK_SCORE VINTAGE_DATE  MOB  DEL90_FC
0       CDLPIL       500-   2024-09-01   12  0.243486
1       CDLPIL       500-   2024-10-01   12  0.256747
2       CDLPIL       500-   2024-11-01   12  0.256845
3   

In [29]:
act_H

,PRODUCT_TYPE,RISK_SCORE,VINTAGE_DATE,MOB,DEL90_ACT
0,CDLPIL,500-,2023-01-01,12,0.167500
1,CDLPIL,500-,2023-02-01,12,0.254235
2,CDLPIL,500-,2023-03-01,12,0.259826
3,CDLPIL,500-,2023-04-01,12,0.264098
4,CDLPIL,500-,2023-05-01,12,0.312721
...,...,...,...,...,...
117,CDLPIL,OTHERS,2024-05-01,12,0.066460
118,CDLPIL,OTHERS,2024-06-01,12,0.057821
119,CDLPIL,OTHERS,2024-07-01,12,0.053463
120,CDLPIL,OTHERS,2024-08-01,12,0.058186


In [30]:
fc_H

,PRODUCT_TYPE,RISK_SCORE,VINTAGE_DATE,MOB,DEL90_FC
0,CDLPIL,500-,2024-09-01,12,0.243486
1,CDLPIL,500-,2024-10-01,12,0.256747
2,CDLPIL,500-,2024-11-01,12,0.256845
3,CDLPIL,500-,2024-12-01,12,0.271897
4,CDLPIL,500-,2025-01-01,12,0.253187
...,...,...,...,...,...
73,CDLPIL,OTHERS,2025-05-01,12,0.066203
74,CDLPIL,OTHERS,2025-06-01,12,0.079122
75,CDLPIL,OTHERS,2025-07-01,12,0.087898
76,CDLPIL,OTHERS,2025-08-01,12,0.100756


In [31]:
import pandas as pd
import numpy as np
from typing import Dict, Tuple

from src.rollrate.forecast import forecast_vintage
from src.config import CFG


def forecast_full_history(
    df_raw: pd.DataFrame,
    matrices_by_mob: Dict,
    max_mob: int = 36,
) -> pd.DataFrame:
    """
    Backtest full-history forecast từ MOB0
    cho tất cả vintage có trong df_raw.

    Output: long-format forecast
        PRODUCT_TYPE, RISK_SCORE, VINTAGE_DATE, MOB, [EAD buckets...]
    """

    orig_col = CFG["orig_date"]

    records = []

    # group theo (product, score, vintage)
    grouped = df_raw.groupby(["PRODUCT_TYPE", "RISK_SCORE", orig_col])

    for (prod, score, vintage), df_seg in grouped:

        try:
            # dùng forecast_vintage để forecast từ MOB0 → max_mob
            fc_dict = forecast_vintage(
                df_raw=df_raw,
                matrices_by_mob=matrices_by_mob,
                product=prod,
                score=score,
                vintage_date=vintage,
                max_mob=max_mob,
                enable_macro=False,
                df_vintage=df_seg,   # subset optimization
            )

            # convert dict → records
            for mob, series in fc_dict.items():
                rec = {
                    "PRODUCT_TYPE": prod,
                    "RISK_SCORE": score,
                    "VINTAGE_DATE": vintage,
                    "MOB": mob,
                }
                rec.update(series.to_dict())
                records.append(rec)

        except Exception as e:
            print(f"⚠️ Skip ({prod}, {score}, {vintage}) due to error: {e}")

    df_fc = pd.DataFrame(records)
    df_fc = df_fc.sort_values(
        ["PRODUCT_TYPE", "RISK_SCORE", "VINTAGE_DATE", "MOB"]
    )

    return df_fc


In [32]:
df_lifecycle_metrics   # actual
df_forecast_hist_long  # FULL BACKTEST


,PRODUCT_TYPE,RISK_SCORE,VINTAGE_DATE,MOB,DPD0,DPD1+,DPD30+,DPD60+,DPD90+,DPD120+,...,PREPAY,WRITEOFF,SOLDOUT,DISB_TOTAL,DEL30_AMT,DEL60_AMT,DEL90_AMT,DEL30_PCT,DEL60_PCT,DEL90_PCT
0,CDLPIL,500-,2023-01-01,24,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,81.127901,0.0,498.44461,81.127901,81.127901,81.127901,0.162762,0.162762,0.162762
1,CDLPIL,500-,2023-01-01,25,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,81.127901,0.0,498.44461,81.127901,81.127901,81.127901,0.162762,0.162762,0.162762
2,CDLPIL,500-,2023-01-01,26,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,81.127901,0.0,498.44461,81.127901,81.127901,81.127901,0.162762,0.162762,0.162762
3,CDLPIL,500-,2023-01-01,27,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,81.127901,0.0,498.44461,81.127901,81.127901,81.127901,0.162762,0.162762,0.162762
4,CDLPIL,500-,2023-01-01,28,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,81.127901,0.0,498.44461,81.127901,81.127901,81.127901,0.162762,0.162762,0.162762
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14266,XSELL,D,2025-09-01,32,8866.916433,1023.133356,1204.530500,773.769413,2654.804998,0.0,...,337.005047,17454.177311,0.0,32599.55940,22087.282221,20882.751721,20108.982309,0.677533,0.640584,0.616848
14267,XSELL,D,2025-09-01,33,8399.937543,970.399372,1145.641865,738.474640,2615.419185,0.0,...,358.839527,18085.624926,0.0,32599.55940,22585.160616,21439.518751,20701.044111,0.692806,0.657663,0.635010
14268,XSELL,D,2025-09-01,34,7957.605542,920.017804,1088.156838,702.999298,2563.443978,0.0,...,379.532803,18702.580794,0.0,32599.55940,23057.180909,21969.024070,21266.024773,0.707285,0.673906,0.652341
14269,XSELL,D,2025-09-01,35,7538.599605,872.023713,1032.637575,668.116256,2501.016858,0.0,...,399.141942,19302.801108,0.0,32599.55940,23504.571797,22471.934222,21803.817966,0.721009,0.689332,0.668838


In [33]:
from src.rollrate.calibration import compute_k_per_product_ifrs

k_dict = compute_k_per_product_ifrs(
    df_actual_lifecycle=df_lifecycle_metrics,
    df_full_forecast=df_forecast_hist_long,
    default_anchor_mob=24,
)


In [34]:
print(k_dict)

{'CDLPIL': 1.0, 'SALPIL': 1.0, 'SPLPIL': 1.0, 'TOPUP': 1.0, 'TWLPIL': 1.0, 'XSELL': 1.0}


In [36]:
from src.rollrate.calibration import apply_k_to_lifecycle, apply_k_to_sale_plan


ImportError: cannot import name 'apply_k_to_lifecycle' from 'src.rollrate.calibration' (C:\Users\MAFC4709\Python_work\RR_model\src\rollrate\calibration.py)

In [ ]:
from src.rollrate.forecast_full_history import forecast_full_history
from src.rollrate.lifecycle import add_del_metrics
from src.rollrate.calibration import compute_k_per_product_ifrs_fullhistory
from src.rollrate.calibration import apply_k_to_lifecycle, apply_k_to_sale_plan

# ================================
# 1) Tính lifecycle actual
# ================================
df_lifecycle = ...    # lifecycle historical bạn đã có
df_lifecycle = add_del_metrics(df_lifecycle, df_raw=df)
df_lifecycle["IS_FORECAST"] = 0
print("✔ Actual lifecycle metrics:", df_lifecycle.shape)

# ================================
# 2) Full-history forecast (BACKTEST)
# ================================
df_fc_hist = forecast_full_history(df_raw=df,
                                   matrices_by_mob=matrices_by_mob,
                                   max_mob=36)
df_fc_hist = add_del_metrics_simple(df_fc_hist)
df_fc_hist["IS_FORECAST"] = 1
print("✔ Full-history forecast:", df_fc_hist.shape)

# ================================
# 3) Tính hệ số k PER PRODUCT
# ================================
k_dict = compute_k_per_product_ifrs_fullhistory(
    df_actual=df_lifecycle,
    df_forecast=df_fc_hist,
    min_obs=5,
    default_anchor_mob=24,
    method="trimmed_mean"
)

print("\n====================== k per product ======================")
print(k_dict)

# ================================
# 4) Áp k vào lifecycle forecast
# ================================
df_lifecycle_calib = apply_k_to_lifecycle(df_lifecycle, k_dict)
print("✔ Applied k to lifecycle forecast")

# ================================
# 5) Áp k vào SALE PLAN
# ================================
df_plan_fc = forecast_sale_plan_by_mob(...)
df_plan_fc = add_del_metrics_simple(df_plan_fc)
df_plan_fc = apply_k_to_sale_plan(df_plan_fc, k_dict)
print("✔ Applied k to sale plan forecast")


## Final 1

In [10]:
import pandas as pd
import numpy as np

from src.config import CFG, BUCKETS_CANON

from src.rollrate.transition import compute_transition_by_mob
from src.rollrate.forecast import forecast_all_vintages
from src.rollrate.forecast_plan import forecast_sale_plan_by_mob

from src.rollrate.lifecycle import (
    get_actual_all_vintages_amount,
    combine_all_lifecycle_amount,
    lifecycle_to_long_df_amount,
    tag_forecast_rows_amount,
    add_del_metrics,
    aggregate_to_product,
    aggregate_products_to_portfolio,
    export_lifecycle_all_products_one_file,
    extend_actual_info_with_portfolio,
    BUCKETS_30P, BUCKETS_60P, BUCKETS_90P,
)

from src.rollrate.calibration import (
    compute_k_per_product_ifrs_fullhistory,
    apply_k_to_lifecycle,
    apply_k_to_sale_plan,
    forecast_full_history
)

from src.rollrate.seasonality import (
    build_seasonality,
    apply_seasonality_to_lifecycle,
    apply_seasonality_to_sale_plan,
)


# ============================================================
# 0) LOAD DATA
# ============================================================

# Giả sử bạn đã có df raw:
# df = pd.read_parquet(...), hoặc từ Oracle, hoặc Excel
# Ở đây giả định biến df đã sẵn có trong session.

# Sale plan
sale_plan_df = pd.read_excel("C:/Users/User/RR_model/sale_plan_5yF2.xlsx", sheet_name="SALE_PLAN")
# Ví dụ: cần các cột: PRODUCT_TYPE, RISK_SCORE, VINTAGE_DATE, EAD_PLAN hoặc DISB_PLAN

# Chuẩn hóa datetime
orig_col = CFG["orig_date"]
df[orig_col] = pd.to_datetime(df[orig_col])

if "VINTAGE_DATE" in sale_plan_df.columns:
    sale_plan_df["VINTAGE_DATE"] = pd.to_datetime(sale_plan_df["VINTAGE_DATE"])


# ============================================================
# 1) TRANSITION MATRICES
# ============================================================

matrices_by_mob, parent_fallback = compute_transition_by_mob(df)
print("✔ matrices_by_mob built.")


# ============================================================
# 2) LIFECYCLE HISTORICAL: ACTUAL + FORECAST (BOOK HIỆN HỮU)
# ============================================================

# 2.1 Forecast all vintages (EAD theo state)
forecast_results = forecast_all_vintages(
    df_raw=df,
    matrices_by_mob=matrices_by_mob,
    max_mob=29,
    enable_macro=False,
)

# 2.2 Actual lifecycle (EAD theo state)
actual_results = get_actual_all_vintages_amount(df)

# 2.3 Combine actual + forecast theo segment
lifecycle_dict = combine_all_lifecycle_amount(actual_results, forecast_results)

# 2.4 Long format (EAD theo bucket)
df_lifecycle = lifecycle_to_long_df_amount(lifecycle_dict)

# 2.5 Tag forecast rows = 1 nếu MOB > max actual MOB per cohort
df_lifecycle = tag_forecast_rows_amount(df_lifecycle, df_raw=df)

# 2.6 Tính DEL30/60/90 + DISB_TOTAL, DELxx_PCT
df_lifecycle = add_del_metrics(df_lifecycle, df_raw=df)
print("✔ Lifecycle (actual + forecast):", df_lifecycle.shape)


# ============================================================
# 3) FULL-HISTORY FORECAST (BACKTEST) CHO CALIBRATION
# ============================================================

df_fc_full = forecast_full_history(
    df_raw=df,
    matrices_by_mob=matrices_by_mob,
    max_mob=36,
)

# Gắn DISB_TOTAL cho full-history forecast bằng DISB per cohort
disp_col = CFG["disb"]

cohort_disb = (
    df.groupby(["PRODUCT_TYPE", "RISK_SCORE", orig_col])[disp_col]
    .sum()
    .rename("DISB_TOTAL")
    .reset_index()
    .rename(columns={orig_col: "VINTAGE_DATE"})
)

df_fc_full = df_fc_full.merge(
    cohort_disb,
    on=["PRODUCT_TYPE", "RISK_SCORE", "VINTAGE_DATE"],
    how="left",
)

# Tính DEL90_PCT từ EAD buckets / DISB_TOTAL
df_fc_full["DEL90_AMT"] = df_fc_full[["DPD90+", "DPD120+", "DPD180+", "WRITEOFF"]].sum(axis=1)
df_fc_full["DEL90_PCT"] = df_fc_full["DEL90_AMT"] / df_fc_full["DISB_TOTAL"]

print("✔ Full-history forecast for calibration:", df_fc_full.shape)


# ============================================================
# 4) CALIBRATION: k PER PRODUCT (IFRS STYLE, ANCHOR MOB)
# ============================================================

# df_lifecycle đã có DEL90_PCT + IS_FORECAST (0/1) sau add_del_metrics + tag_forecast_rows_amount
# compute_k_per_product_ifrs sẽ dùng df_lifecycle (actual) và df_fc_full (full-history forecast)

k_dict = compute_k_per_product_ifrs_fullhistory(
    df_actual_lifecycle=df_lifecycle,
    df_full_forecast=df_fc_full,
    default_anchor_mob=24,
    min_obs=5,
    method="trimmed_mean",
    trim_prop=0.2,
)

print("✔ k per product:", k_dict)


# ============================================================
# 5) ÁP K CHO LIFECYCLE HISTORICAL (CHỈ FORECAST PHẦN TỪ MOB H_APPLY)
# ============================================================

df_lifecycle_calib = apply_k_to_lifecycle(
    df_lifecycle=df_lifecycle,
    k_dict=k_dict,
    default_apply_mob=24,
)

print("✔ Applied k to lifecycle (book forecast).")


# ============================================================
# 6) SALE PLAN FORECAST + CALIBRATION
# ============================================================

# 6.1 Chuẩn hóa RISK_SCORE sale plan (nếu cần)
def normalize_score(x):
    if pd.isna(x):
        return "ALL"
    s = str(x).strip().upper()
    if s in ["", "NA", "NAN", "NONE", "NULL"]:
        return "ALL"
    return s

sale_plan_df["RISK_SCORE"] = sale_plan_df["RISK_SCORE"].apply(normalize_score)

# 6.2 Forecast sale plan từ MOB0 → mob_target
sale_plan_fc = forecast_sale_plan_by_mob(
    sale_plan_df=sale_plan_df,
    matrices_by_mob=matrices_by_mob,
    parent_fallback=parent_fallback,
    mob_target=24,
    start_state=BUCKETS_CANON[0],
    states=list(BUCKETS_CANON),
)

# 6.3 Gắn DISB_TOTAL cho SALE PLAN
#    Giả định sale_plan_df có cột EAD_PLAN hoặc DISB_PLAN; ở đây dùng EAD_PLAN
if "DISB_PLAN" in sale_plan_df.columns:
    plan_disp_col = "DISB_PLAN"
else:
    plan_disp_col = "EAD_PLAN"  # fallback

plan_disb = (
    sale_plan_df.groupby(["PRODUCT_TYPE", "RISK_SCORE", "VINTAGE_DATE"])[plan_disp_col]
    .sum()
    .rename("DISB_TOTAL")
    .reset_index()
)

sale_plan_fc = sale_plan_fc.merge(
    plan_disb,
    on=["PRODUCT_TYPE", "RISK_SCORE", "VINTAGE_DATE"],
    how="left",
)

# 6.4 Tính DEL metrics cho SALE PLAN
sale_plan_fc["DEL30_AMT"] = sale_plan_fc[BUCKETS_30P].sum(axis=1)
sale_plan_fc["DEL60_AMT"] = sale_plan_fc[BUCKETS_60P].sum(axis=1)
sale_plan_fc["DEL90_AMT"] = sale_plan_fc[BUCKETS_90P].sum(axis=1)

sale_plan_fc["DEL30_PCT"] = sale_plan_fc["DEL30_AMT"] / sale_plan_fc["DISB_TOTAL"]
sale_plan_fc["DEL60_PCT"] = sale_plan_fc["DEL60_AMT"] / sale_plan_fc["DISB_TOTAL"]
sale_plan_fc["DEL90_PCT"] = sale_plan_fc["DEL90_AMT"] / sale_plan_fc["DISB_TOTAL"]

sale_plan_fc["IS_FORECAST"] = 1
sale_plan_fc["SOURCE"] = "SALE_PLAN"

# 6.5 Áp k cho sale plan (tất cả MOB đều là forecast)
sale_plan_fc_calib = apply_k_to_sale_plan(
    df_plan_fc=sale_plan_fc,
    k_dict=k_dict,
)

print("✔ Sale plan forecast + calibration:", sale_plan_fc_calib.shape)


# ============================================================
# 7) (OPTIONAL) SEASONALITY — CHỈ ÁP DỤNG CHO FORECAST
# ============================================================

# Xây seasonality từ actual-only
df_actual_only = df_lifecycle_calib[df_lifecycle_calib["IS_FORECAST"] == 0].copy()
seasonality = build_seasonality(df_actual_only)

# Áp seasonality cho phần forecast của lifecycle
df_lc_for_season = df_lifecycle_calib[df_lifecycle_calib["IS_FORECAST"] == 1].copy()
df_lc_for_season = apply_seasonality_to_lifecycle(df_lc_for_season, seasonality)

# Giữ nguyên actual (không seasonality)
df_lc_actual = df_lifecycle_calib[df_lifecycle_calib["IS_FORECAST"] == 0].copy()

df_lifecycle_final = pd.concat(
    [df_lc_actual, df_lc_for_season],
    ignore_index=True
)

# Áp seasonality cho sale plan
sale_plan_fc_final = apply_seasonality_to_sale_plan(sale_plan_fc_calib, seasonality)

print("✔ Seasonality applied (forecast only).")


# ============================================================
# 8) GOM LIFECYCLE HISTORICAL + SALE PLAN
# ============================================================

df_lifecycle_all = pd.concat(
    [df_lifecycle_final, sale_plan_fc_final],
    ignore_index=True
)

print("✔ Combined lifecycle + sale plan:", df_lifecycle_all.shape)


# ============================================================
# 9) AGGREGATE PRODUCT + PORTFOLIO
# ============================================================

df_prod = aggregate_to_product(df_lifecycle_all)

df_port = aggregate_products_to_portfolio(
    df_del_prod=df_prod,
    portfolio_name="PORTFOLIO_ALL",
)

df_del_prod_all = pd.concat([df_prod, df_port], ignore_index=True)
print("✔ Aggregated product + portfolio:", df_del_prod_all.shape)


# ============================================================
# 🔟 COMPUTE actual_info (cohort max actual MOB) CHO EXPORT
# ============================================================

actual_info_prod = (
    df_lifecycle_all[df_lifecycle_all["IS_FORECAST"] == 0]
    .groupby(["PRODUCT_TYPE", "VINTAGE_DATE"])["MOB"]
    .max()
    .to_dict()
)

actual_info_all = extend_actual_info_with_portfolio(
    actual_info_prod,
    portfolio_name="PORTFOLIO_ALL",
)


# ============================================================
# 11) EXPORT EXCEL
# ============================================================

export_lifecycle_all_products_one_file(
    df_del_prod=df_del_prod_all,
    actual_info=actual_info_all,
    filename="Lifecycle_All_Products_Final.xlsx"
)

print("✔✔✔ EXPORT DONE → Lifecycle_All_Products_Final.xlsx")


ImportError: cannot import name 'apply_k_to_lifecycle' from 'src.rollrate.calibration' (C:\Users\User\RR_model\src\rollrate\calibration.py)